# Sequence modeling for ranking task

# Set up

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import os
import sys

import lightning as L
import numpy as np
import pandas as pd
import torch
from dotenv import load_dotenv
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from mlflow.exceptions import MlflowException
from mlflow.models.signature import infer_signature
from pydantic import BaseModel
from torch.utils.data import DataLoader

import mlflow

sys.path.insert(0, "..")

from src.dataset import UserItemRatingDFDataset
from src.eval.compare_runs import ModelMetricsComparisonVisualizer
from src.id_mapper import IDMapper
from src.sequence.inference import SequenceRatingPredictionInferenceWrapper
from src.sequence.model import SequenceRatingPrediction
from src.sequence.trainer import LitSequenceRatingPrediction
from src.sequence.utils import generate_item_sequences
from src.viz import custom_style_plotly

load_dotenv()
custom_style_plotly()

/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Controller

In [3]:
# This is a parameter cell used by papermill
max_epochs = 100

In [4]:
class Args(BaseModel):
    testing: bool = False
    author: str = "quy.dinh"
    log_to_mlflow: bool = True
    experiment_name: str = "Retriever"
    run_name: str = "002-sequence-model-increase-lr"
    notebook_persist_dp: str = None
    random_seed: int = 41
    device: str = None

    max_epochs: int = max_epochs
    batch_size: int = 128

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    embedding_dim: int = 128
    dropout: float = 0.3
    early_stopping_patience: int = 5
    learning_rate: float = 0.003
    l2_reg: float = 1e-5

    mlf_item2vec_model_name: str = "item2vec"
    mlf_model_name: str = "sequence_rating_prediction"
    min_metric_value: float = 0.05

    best_checkpoint_path: str = None

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            logger.warning(
                "Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if self.device is None:
            self.device = (
                "cuda"
                if torch.cuda.is_available()
                else "mps" if torch.backends.mps.is_available() else "cpu"
            )

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2025-03-08 14:55:28.722 | INFO     | __main__:init:47 - Setting up MLflow experiment Retriever - run 002-sequence-model-increase-lr...


{
  "testing": false,
  "author": "quy.dinh",
  "log_to_mlflow": true,
  "experiment_name": "Retriever",
  "run_name": "002-sequence-model-increase-lr",
  "notebook_persist_dp": "/home/dvq/frostmourne/recsys-blog/1-seq-model/notebooks/data/002-sequence-model-increase-lr",
  "random_seed": 41,
  "device": "cuda",
  "max_epochs": 100,
  "batch_size": 128,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "embedding_dim": 128,
  "dropout": 0.3,
  "early_stopping_patience": 5,
  "learning_rate": 0.003,
  "l2_reg": 0.00001,
  "mlf_item2vec_model_name": "item2vec",
  "mlf_model_name": "sequence_rating_prediction",
  "min_metric_value": 0.05,
  "best_checkpoint_path": null
}


# Implement

In [5]:
def init_model(n_users, n_items, embedding_dim, dropout, item_embedding=None):
    model = SequenceRatingPrediction(
        n_users, n_items, embedding_dim, dropout=dropout, item_embedding=item_embedding
    )
    return model

# Test implementation

In [6]:
embedding_dim = 8
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 4, 5, 3, 0]
item_sequences = [
    [-1, -1, 2, 3],
    [-1, -1, 2, 3],
    [-1, -1, 1, 3],
    [-1, -1, 2, 1],
    [-1, -1, 2, 1],
]

n_users = len(set(user_indices))
n_items = len(set(item_indices))

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
        "item_sequence": item_sequences,
    }
)

model = init_model(n_users, n_items, embedding_dim, args.dropout)

# Example forward pass
model.eval()
user = torch.tensor([0])
item_sequence = torch.tensor([[-1, -1, -1, 0, 1]])
target_item = torch.tensor([2])
predictions = model.predict(user, item_sequence, target_item)
print(predictions)
model.train()

tensor([[0.4716]], grad_fn=<SigmoidBackward0>)


SequenceRatingPrediction(
  (item_embedding): Embedding(6, 8, padding_idx=5)
  (user_embedding): Embedding(3, 8)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc_rating): Sequential(
    (0): Linear(in_features=24, out_features=8, bias=True)
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [7]:
rating_dataset = UserItemRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)

train_loader = DataLoader(
    rating_dataset, batch_size=batch_size, shuffle=False, drop_last=True
)

In [8]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0.0000, 0.8000]), 'item_sequence': tensor([[-1, -1,  2,  3],
        [-1, -1,  2,  3]]), 'item_sequence_ts_bucket': tensor([], size=(2, 0), dtype=torch.int64), 'item_feature': tensor([], size=(2, 0))}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([1.0000, 0.6000]), 'item_sequence': tensor([[-1, -1,  1,  3],
        [-1, -1,  2,  1]]), 'item_sequence_ts_bucket': tensor([], size=(2, 0), dtype=torch.int64), 'item_feature': tensor([], size=(2, 0))}


In [9]:
# model
lit_model = LitSequenceRatingPrediction(model, log_dir=args.notebook_persist_dp)

# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persist_dp}/test",
    max_epochs=2,
    accelerator=args.device if args.device else "auto",
)
trainer.fit(
    model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 SUPER') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params | Mode 
-----------------------------------------------------------
0 | model | SequenceRatingPrediction | 297    | train
-----------------------------------------------------------
297       Trainable params
0         Non-trainable params
297       Total params
0.001     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|                                                                                                                                                               | 0/2 [00:00<?, ?it/s]

/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.36it/s, v_num=0, train_loss_step=0.182]
idation: |                                                                                                                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 456.70it/s, v_num=0, train_loss_step=0.161, val_loss=0.145, train_loss_epoch=0.178]
idation: |                                                                                                                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 198.6

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 163.35it/s, v_num=0, train_loss_step=0.161, val_loss=0.144, train_loss_epoch=0.175]


In [10]:
users = torch.tensor([0, 0, 0, 0])
item_sequences = torch.tensor(
    [[-1, -1, 2, 3], [-1, -1, 2, 3], [-1, -1, 1, 3], [-1, -1, 2, 1]]
)
items = torch.tensor([0, 1, 2, 3])
predictions = model.predict(users, item_sequences, items)
print(predictions)

tensor([[0.4761],
        [0.5818],
        [0.5557],
        [0.3907]], grad_fn=<SigmoidBackward0>)


In [11]:
def create_predict_df(
    train_df,
    val_user_indices,
    val_timestamp,
    rating_col,
    timestamp_col,
    sequence_length=10,
):
    predict_df = pd.DataFrame(
        {
            "user_indice": val_user_indices,
            "item_indice": -1,  # placeholder
            "timestamp": val_timestamp,
            "source": "predict",
        }
    )

    predict_df = (
        pd.concat(
            [
                train_df.loc[lambda df: df[rating_col].gt(0)][
                    ["user_indice", "item_indice", timestamp_col]
                ].assign(source="train"),
                predict_df,
            ],
            axis=0,
        )
        .pipe(
            generate_item_sequences,
            "user_indice",
            "item_indice",
            timestamp_col,
            sequence_length=sequence_length,
            padding=True,
            padding_value=-1,
        )
        .loc[lambda df: df["source"].eq("predict")]
        .assign(item_sequence=lambda df: df["item_sequence"].apply(np.array))
    )

    return predict_df


predict_df = create_predict_df(
    train_df,
    user_indices,
    timestamps[-1],
    args.rating_col,
    args.timestamp_col,
    sequence_length=10,
)

predict_df

,user_indice,item_indice,timestamp,source,item_sequence
0,0,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1]"
1,0,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1]"
2,1,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2]"
3,2,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3]"
4,2,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3]"


In [12]:
recommendations = model.recommend(
    torch.tensor(predict_df["user_indice"].values),
    torch.tensor(predict_df["item_sequence"].values.tolist()),
    k=2,
    batch_size=4,
)
recommendations

/tmp/ipykernel_32745/349226613.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  torch.tensor(predict_df["item_sequence"].values.tolist()),
Generating recommendations: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2713.00it/s]


{'user_indice': [0, 0, 0, 0, 1, 1, 2, 2, 2, 2],
 'recommendation': [5, 1, 5, 1, 0, 2, 0, 1, 0, 1],
 'score': [0.4957159161567688,
  0.48693835735321045,
  0.4957159161567688,
  0.48693835735321045,
  0.5202603340148926,
  0.5167825818061829,
  0.5595028400421143,
  0.556573212146759,
  0.5595028400421143,
  0.556573212146759]}

# Prep data

In [13]:
train_df = pd.read_parquet("../data/train_features.parquet")
val_df = pd.read_parquet("../data/val_features.parquet")
idm_fp = "../data/idm.json"
idm = IDMapper().load(idm_fp)

assert (
    train_df[args.user_col].map(lambda s: idm.get_user_index(s))
    != train_df["user_indice"]
).sum() == 0, "Mismatch IDM"
assert (
    val_df[args.user_col].map(lambda s: idm.get_user_index(s)) != val_df["user_indice"]
).sum() == 0, "Mismatch IDM"

In [14]:
user_indices = train_df["user_indice"].unique()
item_indices = train_df["item_indice"].unique()

logger.info(f"{len(user_indices)=:,.0f}, {len(item_indices)=:,.0f}")

2025-03-08 14:55:29.436 | INFO     | __main__:<module>:4 - len(user_indices)=19,734, len(item_indices)=7,388


In [15]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AE224PFXAEAT66IXX43GRJSWHXCA,0399159312,2.0,1373291889000,6822,4732,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
1,AE224PFXAEAT66IXX43GRJSWHXCA,B000FA5TTW,1.0,1382077065000,6822,1581,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
2,AE224PFXAEAT66IXX43GRJSWHXCA,030758836X,1.0,1424138603000,6822,2712,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
3,AE224PFXAEAT66IXX43GRJSWHXCA,B00MSRW6SM,4.0,1437924147000,6822,4217,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 473..."
4,AE224PFXAEAT66IXX43GRJSWHXCA,B00A18VD7A,1.0,1464603674000,6822,6558,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 4732.0, 1..."
...,...,...,...,...,...,...,...
194419,AHZZZ6UASY7CGOTGP5BH5637FMPA,B017GFRJZK,5.0,1508089337653,6916,6998,"[-1.0, -1.0, -1.0, -1.0, -1.0, 4088.0, 2670.0,..."
194420,AHZZZ6UASY7CGOTGP5BH5637FMPA,B00RPM9MJ6,4.0,1521230143557,6916,5013,"[-1.0, -1.0, -1.0, -1.0, 4088.0, 2670.0, 6456...."
194421,AHZZZ6UASY7CGOTGP5BH5637FMPA,B00RU7SNP0,4.0,1534867184329,6916,1099,"[-1.0, -1.0, -1.0, 4088.0, 2670.0, 6456.0, 141..."
194422,AHZZZ6UASY7CGOTGP5BH5637FMPA,B00HGSVGSY,4.0,1534867223318,6916,685,"[-1.0, -1.0, 4088.0, 2670.0, 6456.0, 1416.0, 3..."


# Train

In [16]:
rating_dataset = UserItemRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)
val_rating_dataset = UserItemRatingDFDataset(
    val_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [17]:
n_items = len(item_indices)
n_users = len(user_indices)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)

#### Predict before train

In [18]:
model.item_embedding

Embedding(7389, 128, padding_idx=7388)

In [19]:
val_df = val_rating_dataset.df
val_df.sample(10)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
432,AEJVCG5LPO4AITZXXLCT6A2ILDRA,B07S8K7YR5,1.0,1638942452082,11502,2766,"[1945, 2316, 1176, 144, 1996, 2675, 1794, 4533..."
2084,AGILJFVEHJX5W3Q7QYJZ7MRVU6ZQ,B07NXPN3B8,1.0,1650005449499,14381,2580,"[6332, 4637, 768, 5118, 4286, 1286, 1547, 5610..."
638,AERWHGDBA6B3XDADBXDKSBQZOW4A,B07BDF7WKP,1.0,1642612010634,14781,4994,"[1607, 534, 705, 2460, 5209, 5357, 4039, 4093,..."
1567,AFVGMVRC6BY5MNOAAJP5RDTAUZHA,B07Y8CXZ2Y,0.8,1635187214051,18838,4861,"[6904, 3948, 2462, 731, 1257, 1176, 3773, 6966..."
2277,AGNAIJAGMTYLXGUD5HJAZNCT6CHQ,B01MYDJB6R,0.4,1645479684670,10382,2039,"[2722, 3557, 11, 1291, 3225, 6331, 691, 230, 3..."
1697,AG23XM5L57T7VFA6EWXUOV5ENZCQ,B0043M6L0Y,1.0,1635397949880,10278,7131,"[-1, -1, -1, 5333, 5581, 4523, 591, 5941, 2499..."
550,AEOOSM6JAO36HW4LRRTZW6RD7BPA,B07TQKF1RS,0.4,1650022591533,2280,2227,"[-1, -1, -1, -1, 334, 6573, 4473, 3576, 2812, ..."
426,AEJVCG5LPO4AITZXXLCT6A2ILDRA,B08BYC481D,0.6,1634255015008,11502,1996,"[3324, 3291, 2225, 5885, 6966, 5614, 1945, 231..."
3254,AHPKVQKIOHKZXIFNK5KNBWNHWXVQ,B076H5KB7W,0.6,1643029153491,15735,7069,"[2036, 5862, 3186, 1720, 6013, 2318, 1596, 388..."
297,AEEI25ROY2PE3LJVDFU2OWY7D2TA,B08SNVSF47,1.0,1628762894386,4064,1794,"[1996, 1409, 1589, 2671, 7292, 6427, 6964, 612..."


In [20]:
user_id = val_df.sample(1)[args.user_col].values[0]
# user_id = "AH4AOFTTDPHPAFAAVFMAF25H2LIQ"
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
with pd.option_context("display.max_colwidth", None):
    display(test_df)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
2559,AGVLWLNPXW5GHREKE5ZADVELHKDA,B010JG1YFO,0.8,1629681042987,5758,6483,"[5249, 824, 7382, 6226, 1788, 2321, 3806, 1708, 3842, 506]"
2560,AGVLWLNPXW5GHREKE5ZADVELHKDA,B00YTXTIDO,0.8,1632081260378,5758,3278,"[824, 7382, 6226, 1788, 2321, 3806, 1708, 3842, 506, 6483]"
2561,AGVLWLNPXW5GHREKE5ZADVELHKDA,B004VMV4AI,0.4,1640128845041,5758,2342,"[7382, 6226, 1788, 2321, 3806, 1708, 3842, 506, 6483, 3278]"
2562,AGVLWLNPXW5GHREKE5ZADVELHKDA,B00805582W,0.2,1640205802594,5758,1614,"[6226, 1788, 2321, 3806, 1708, 3842, 506, 6483, 3278, 2342]"
2563,AGVLWLNPXW5GHREKE5ZADVELHKDA,B07ZJ2VHBB,0.2,1649269450269,5758,4442,"[1788, 2321, 3806, 1708, 3842, 506, 6483, 3278, 2342, 1614]"


In [21]:
test_row = test_df.loc[lambda df: df[args.rating_col].gt(0)].iloc[0]
item_id = test_row[args.item_col]
item_sequence = test_row["item_sequence"]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)
user = torch.tensor([user_indice])
item_sequence = torch.tensor([item_sequence])
item = torch.tensor([item_indice])

model.eval()
model.predict(user, item_sequence, item)
model.train()

2025-03-08 14:55:29.637 | INFO     | __main__:<module>:4 - Test predicting before training with user_id = AGVLWLNPXW5GHREKE5ZADVELHKDA and parent_asin = B010JG1YFO


SequenceRatingPrediction(
  (item_embedding): Embedding(7389, 128, padding_idx=7388)
  (user_embedding): Embedding(19734, 128)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc_rating): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

#### Training loop

##### Overfit 1 batch

In [22]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, dropout=0)
lit_model = LitSequenceRatingPrediction(
    model,
    learning_rate=args.learning_rate,
    l2_reg=0.0,
    log_dir=args.notebook_persist_dp,
    accelerator=args.device,
)

log_dir = f"{args.notebook_persist_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=100,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params | Mode 
-----------------------------------------------------------
0 | model | SequenceRatingPrediction | 3.5 M  | train
-----------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
14.086    Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.
/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.
/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.

Epoch 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 112.29it/s, v_num=0, train_loss_step=0.188]
idation: |                                                                                                                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 148.68it/s, v_num=0, train_loss_step=0.113, val_loss=0.123, train_loss_epoch=0.188]
idation: |                                                                                                                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 147.92i

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.25it/s, v_num=0, train_loss_step=4.37e-5, val_loss=4.67e-5, train_loss_epoch=4.37e-5]


2025-03-08 14:55:36.750 | INFO     | __main__:<module>:29 - Logs available at /home/dvq/frostmourne/recsys-blog/1-seq-model/notebooks/data/002-sequence-model-increase-lr/logs/overfit/lightning_logs/version_0


In [23]:
# Need to make sure port 6006 at local is accessible
%tensorboard --logdir $trainer.log_dir

##### Fit on all data

In [24]:
# papermill_description=fit-model
early_stopping = EarlyStopping(
    monitor="val_loss", patience=args.early_stopping_patience, mode="min", verbose=False
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persist_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_loss",
    mode="min",
)

model = init_model(
    n_users,
    n_items,
    args.embedding_dim,
    dropout=args.dropout
)
lit_model = LitSequenceRatingPrediction(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persist_dp,
    evaluate_ranking=True,
    idm=idm,
    args=args,
    accelerator=args.device,
    checkpoint_callback=checkpoint_callback,
)

log_dir = f"{args.notebook_persist_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    accelerator=args.device if args.device else "auto",
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params | Mode 
-----------------------------------------------------------
0 | model | SequenceRatingPrediction | 3.5 M  | train
-----------------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
14.086    Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/dvq/frostmourne/recsys-blog/1-seq-model/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1518/1518 [00:08<00:00, 171.30it/s, v_num=bf67, train_loss_step=0.0402]
idation: |                                                                                                                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1518/1518 [00:08<00:00, 177.82it/s, v_num=bf67, train_loss_step=0.0426, val_loss=0.0354, train_loss_epoch=0.0383]
idation: |                                                                                                                                                                                     | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1518/1518 [00:08<00:00, 172.78it/s,

2025-03-08 14:56:49.519 | INFO     | src.sequence.trainer:on_fit_end:121 - Loading best model from /home/dvq/frostmourne/recsys-blog/1-seq-model/notebooks/data/002-sequence-model-increase-lr/checkpoints/best-checkpoint.ckpt...
2025-03-08 14:56:49.675 | INFO     | src.sequence.trainer:on_fit_end:129 - Logging ranking metrics...
Generating recommendations: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 434/434 [00:00<00:00, 607.29it/s]


🏃 View run 002-sequence-model-increase-lr at: http://localhost:5002/#/experiments/1/runs/a87d7cc769c5411280fe6811b6dcbf67
🧪 View experiment at: http://localhost:5002/#/experiments/1


In [25]:
logger.info(
    f"Test predicting after training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.eval()
model = model.to(user.device)
model.predict(user, item_sequence, item)
model.train()

2025-03-08 14:56:54.579 | INFO     | __main__:<module>:1 - Test predicting after training with user_id = AGVLWLNPXW5GHREKE5ZADVELHKDA and parent_asin = B010JG1YFO


SequenceRatingPrediction(
  (item_embedding): Embedding(7389, 128, padding_idx=7388)
  (user_embedding): Embedding(19734, 128)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc_rating): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

# Load best checkpoint

In [26]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = LitSequenceRatingPrediction.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    model=init_model(n_users, n_items, args.embedding_dim, dropout=0),
)

2025-03-08 14:56:54.611 | INFO     | __main__:<module>:1 - Loading best checkpoint from /home/dvq/frostmourne/recsys-blog/1-seq-model/notebooks/data/002-sequence-model-increase-lr/checkpoints/best-checkpoint.ckpt...


In [27]:
best_model = best_trainer.model.to(lit_model.device)

In [28]:
best_model.eval()
best_model.predict(user, item_sequence, item)
best_model.train()

SequenceRatingPrediction(
  (item_embedding): Embedding(7389, 128, padding_idx=7388)
  (user_embedding): Embedding(19734, 128)
  (relu): ReLU()
  (dropout): Dropout(p=0, inplace=False)
  (fc_rating): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

### Persist id mapping

In [29]:
if args.log_to_mlflow:
    # Persist id_mapping so that at inference we can predict based on item_ids (string) instead of item_index
    run_id = trainer.logger.run_id
    mlf_client = trainer.logger.experiment
    mlf_client.log_artifact(run_id, idm_fp)

### Wrap inference function and register best checkpoint as MLflow model

In [30]:
inferrer = SequenceRatingPredictionInferenceWrapper(best_model)

In [31]:
sample_input = {
    "user_ids": [idm.get_user_id(0)],
    "item_sequences": [[idm.get_item_id(0), idm.get_item_id(1)]],
    "item_ids": [idm.get_item_id(0)],
}
sample_output = inferrer.infer([0], [[0, 1]], [0])
sample_output

array([0.62947756], dtype=float32)

In [32]:
if args.log_to_mlflow:
    run_id = trainer.logger.run_id
    sample_output_np = sample_output
    signature = infer_signature(sample_input, sample_output_np)
    idm_filename = idm_fp.split("/")[-1]
    with mlflow.start_run(run_id=run_id):
        mlflow.pyfunc.log_model(
            python_model=inferrer,
            artifact_path="inferrer",
            # We log the id_mapping to the predict function so that it can accept item_id and automatically convert ot item_indice for PyTorch model to use
            artifacts={"idm": mlflow.get_artifact_uri(idm_filename)},
            signature=signature,
            input_example=sample_input,
            registered_model_name=args.mlf_model_name,
        )

2025/03/08 14:56:54 INFO mlflow.pyfunc: Validating input example against model signature
2025/03/08 14:56:57 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'sequence_rating_prediction' already exists. Creating a new version of this model...
2025/03/08 14:56:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sequence_rating_prediction, version 2


🏃 View run 002-sequence-model-increase-lr at: http://localhost:5002/#/experiments/1/runs/a87d7cc769c5411280fe6811b6dcbf67
🧪 View experiment at: http://localhost:5002/#/experiments/1


Created version '2' of model 'sequence_rating_prediction'.


# Set the newly trained model as champion

In [34]:
if args.log_to_mlflow:
    key_metric = 'val_RecallTopKMetric_at_k_as_step'
    # Get current champion
    deploy_alias = "champion"
    curr_model_run_id = None

    min_metric_value = args.min_metric_value

    try:
        curr_champion_model = mlf_client.get_model_version_by_alias(
            args.mlf_model_name, deploy_alias
        )
        curr_model_run_id = curr_champion_model.run_id
    except MlflowException as e:
        if "not found" in str(e).lower():
            logger.info(
                f"There is no {deploy_alias} alias for model {args.mlf_model_name}"
            )

    # Compare new vs curr models
    new_mlf_run = trainer.logger.experiment.get_run(trainer.logger.run_id)
    new_metrics = new_mlf_run.data.metrics
    metric_value = new_metrics[key_metric]
    if curr_model_run_id:
        curr_model_run_info = mlf_client.get_run(curr_model_run_id)
        curr_metrics = curr_model_run_info.data.metrics
        if (curr_metric_value := curr_metrics[key_metric]) > min_metric_value:
            logger.info(
                f"Current {deploy_alias} model has {curr_metric_value:,.4f} ROC-AUC. Setting it to the deploy baseline..."
            )
            min_metric_value = curr_metric_value

        top_metrics = [key_metric]
        vizer = ModelMetricsComparisonVisualizer(curr_metrics, new_metrics, top_metrics)
        print("Comparing metrics between new run and current champion:")
        display(vizer.compare_metrics_df())
        vizer.create_metrics_comparison_plot(n_cols=5)
        vizer.plot_diff()

    # Register new champion
    if metric_value < min_metric_value:
        logger.info(
            f"Current run has {key_metric} = {metric_value:,.4f}, smaller than {min_metric_value:,.4f}. Skip aliasing this model as the new {deploy_alias}.."
        )
    else:
        logger.info("Aliasing the new model as champion...")
        # Get the model version for current run by assuming it's the most recent registered version
        model_version = (
            mlf_client.get_registered_model(args.mlf_model_name)
            .latest_versions[0]
            .version
        )

        mlf_client.set_registered_model_alias(
            name=args.mlf_model_name, alias="champion", version=model_version
        )

        mlf_client.set_model_version_tag(
            name=args.mlf_model_name,
            version=model_version,
            key="author",
            value=args.author,
        )

2025-03-08 20:13:01.886 | INFO     | __main__:<module>:16 - There is no champion alias for model sequence_rating_prediction
2025-03-08 20:13:01.892 | INFO     | __main__:<module>:42 - Current run has val_RecallTopKMetric_at_k_as_step = 0.0336, smaller than 0.0500. Skip aliasing this model as the new champion..


# Clean up

In [35]:
all_params = [args]

if args.log_to_mlflow:
    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.dict()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

🏃 View run 002-sequence-model-increase-lr at: http://localhost:5002/#/experiments/1/runs/a87d7cc769c5411280fe6811b6dcbf67
🧪 View experiment at: http://localhost:5002/#/experiments/1


/tmp/ipykernel_32745/747004171.py:6: PydanticDeprecatedSince20:

The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/

